In [ ]:
!pip install transformers

In [3]:
import torch
from transformers import BertTokenizer,BertModel
import matplotlib.pyplot as plt
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model=BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,)
model.to("cuda")

In [6]:
import nltk
from nltk.corpus import twitter_samples
nltk.download('twitter_samples')
all_pos_tweets=twitter_samples.strings('positive_tweets.json')
all_neg_tweets=twitter_samples.strings('negative_tweets.json')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


In [7]:
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
stopwords_english=stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def remstop(arr):
  ans=[]
  for sent in arr:
    temp=re.sub("[@#$%^&*{}_\d+]","",sent)
    temp=re.sub(r"http\S+", "", temp)
    temp=temp.split(" ")
    temp=[i+" " for i in temp if i not in stopwords_english]
    temp=("").join(temp)
    ans.append(temp[:-1])
  return ans

In [15]:
all_pos_tweets_process=remstop(all_pos_tweets)
all_neg_tweets_process=remstop(all_neg_tweets)

In [10]:
print(all_pos_tweets[20])
print(all_pos_tweets_process[20])

#FollowFriday @MBandScott_ @Eric_FLE @pointsolutions3 for being top new followers in my community this week :)
FollowFriday MBandScott EricFLE pointsolutions top new followers community week :)


In [16]:
def bertready(arr):
  for i in range(len(arr)):
    arr[i]="[CLS] "+arr[i]+" [SEP]"
  return arr

In [17]:
all_pos_bert=bertready(all_pos_tweets_process)
all_neg_bert=bertready(all_neg_tweets_process)

In [18]:
all_pos_bert[30]

'[CLS] MY kik - hatessuce kik kikme lgbt tinder nsfw akua cumshot :)  [SEP]'

In [19]:
def token_bert(arr):
  ans=[]
  for i in range(len(arr)):
    temp=tokenizer.tokenize(arr[i])
    if len(temp)<30:
      ans.append(temp)
  return ans

all_pos_bert_token=token_bert(all_pos_bert)
all_neg_bert_token=token_bert(all_neg_bert)

In [20]:
max_len=0
for i in all_neg_bert_token:
  max_len=max(len(i),max_len)


for i in all_pos_bert_token:
  max_len=max(len(i),max_len)

print(max_len)

29


In [21]:
def convert(arr):
  tok=[]
  seg=[]
  for token in arr:
    seg.append([1]*len(token)+[0]*(max_len-len(token)))
    tok.append(token+["[PAD]"]*(max_len-len(token)))
  return tok,seg

all_neg_bert_token_padded,segment_neg=convert(all_neg_bert_token)
all_pos_bert_token_padded,segment_pos=convert(all_pos_bert_token)


In [22]:
train_pos=all_pos_bert_token_padded[:2000]
train_neg=all_neg_bert_token_padded[:2000]
test_pos=all_pos_bert_token_padded[4000:]
test_neg=all_neg_bert_token_padded[4000:]

segment_pos_train=segment_pos[:2000]
segment_neg_train=segment_neg[:2000]
segment_pos_test=segment_pos[4000:]
segment_neg_test=segment_neg[4000:]


In [23]:
label=[0]*2000+[1]*2000
train_data=train_pos+train_neg

In [25]:
import numpy as np
train_data_stack=np.vstack(train_data)


In [26]:
label_stack=np.vstack(label)
label_stack.shape

(4000, 1)

In [27]:
seg_train=segment_pos_train+segment_neg_train

In [28]:
segment_stack=np.vstack(seg_train)

In [29]:
indexed_tokens=[tokenizer.convert_tokens_to_ids(i) for i in train_data]


In [30]:
indexed_tokens_stack=np.vstack(indexed_tokens)

In [31]:
tokens_tensor=torch.tensor(indexed_tokens_stack,device=device)
segment_tensor=torch.tensor(segment_stack,device=device)

In [32]:
with torch.no_grad():
  outputs=model(tokens_tensor,segment_tensor)
  hidden_states=outputs[2]

In [33]:
hidden_states[0].shape

torch.Size([4000, 29, 768])

In [34]:
import torch.nn as nn


In [120]:
class SentimentLSTM(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm1=nn.LSTM(768,128,32,dropout=0.3,batch_first=True)
    self.dropout=nn.Dropout(0.3)
    self.fc1=nn.Linear(128,64)
    self.fc2=nn.Linear(64,32)
    self.fc3=nn.Linear(32,1)
    self.relu=nn.ReLU()
    self.sigmoid=nn.Sigmoid()
  
  def forward(self,x,hidden,carry):
    batch_size=x.size(0)
    lstm_out,(hidden,carry)=self.lstm1(x,(hidden,carry))
    out=self.fc1(lstm_out[:,1,:])
    
    out=self.relu(out)
    out=self.fc2(out)
    out=self.relu(out)
    out=self.fc3(out)
    out=self.sigmoid(out)
    print(out.size())
    return out,hidden,carry

  def init_hidden(self,batch_size):
    weight = next(self.parameters()).data
    
    hidden = weight.new(32, batch_size, 128).zero_().cuda()
    carry=weight.new(32, batch_size,128).zero_().cuda()
    return hidden,carry
model_sent=SentimentLSTM()
print(model_sent)
model_sent.to(device)

SentimentLSTM(
  (lstm1): LSTM(768, 128, num_layers=32, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


SentimentLSTM(
  (lstm1): LSTM(768, 128, num_layers=32, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [58]:
rnn = nn.LSTM(768, 128, 5)
input = torch.randn(5, 3, 768)
h0 = torch.randn(5, 3, 128)
c0 = torch.randn(5, 3, 128)
output, (hn, cn) = rnn(input, (h0, c0))

In [63]:
train_data=hidden_states[0]


In [78]:
label_train=torch.tensor(label_stack,device=device)

In [109]:
model_sent=SentimentLSTM()
print(model_sent)
model_sent.to(device)

SentimentLSTM(
  (lstm1): LSTM(768, 128, num_layers=32, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


SentimentLSTM(
  (lstm1): LSTM(768, 128, num_layers=32, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [76]:
label.size()

torch.Size([4000])

In [81]:
from torch.utils.data import DataLoader,TensorDataset

In [94]:
train_data_arr=train_data.cpu().detach().numpy()

In [95]:
label_train_arr=label_train.cpu().detach().numpy()

In [96]:
train_data_arr.shape

(4000, 29, 768)

In [84]:
train_data.shape

torch.Size([4000, 29, 768])

In [97]:
train_load = TensorDataset(torch.from_numpy(train_data_arr),torch.from_numpy(label_train_arr))
train_loader=DataLoader(train_load,shuffle=True,batch_size=16)

In [98]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

In [136]:
class SentimentLSTM(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm1=nn.LSTM(768,128,32,dropout=0.3,batch_first=True)
    self.dropout=nn.Dropout(0.3)
    self.fc1=nn.Linear(128,64)
    self.fc2=nn.Linear(64,32)
    self.fc3=nn.Linear(32,1)
    self.relu1=nn.ReLU()
    self.relu2=nn.ReLU()
    self.sigmoid=nn.Sigmoid()
  
  def forward(self,x,hidden,carry):
    batch_size=x.size(0)
    lstm_out,(hidden,carry)=self.lstm1(x,(hidden,carry))
    out=self.fc1(lstm_out[:,1,:])
    # print(out.size())
    out=self.relu1(out)
    out=self.fc2(out)
    out=self.relu2(out)
    out=self.fc3(out)
    out=self.sigmoid(out)
    # print(out.size())
    return out,hidden,carry

  def init_hidden(self,batch_size):
    weight = next(self.parameters()).data
    
    hidden = weight.new(32, batch_size, 128).zero_().cuda()
    carry=weight.new(32, batch_size,128).zero_().cuda()
    return hidden,carry
model_sent=SentimentLSTM()
print(model_sent)
model_sent.to(device)

SentimentLSTM(
  (lstm1): LSTM(768, 128, num_layers=32, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu1): ReLU()
  (relu2): ReLU()
  (sigmoid): Sigmoid()
)


SentimentLSTM(
  (lstm1): LSTM(768, 128, num_layers=32, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu1): ReLU()
  (relu2): ReLU()
  (sigmoid): Sigmoid()
)

In [141]:
torch.autograd.set_detect_anomaly(True)
lr=0.01
criterion=nn.BCELoss()
optimizer=torch.optim.Adam(model_sent.parameters(),lr=lr)
epoch=4
model_sent.train()
counter=0

for i in range(epoch):
  h,c=model_sent.init_hidden(16)
  for inputs,labels in train_loader:
    counter+=1
    inputs=inputs.cuda()
    labels=labels.cuda()
    model_sent.zero_grad()
    output,h,c=model_sent(inputs,h,c)
    loss=criterion(output.squeeze(),labels.float().squeeze())
    loss.backward(retain_graph=True)
    nn.utils.clip_grad_norm_(model_sent.parameters(), 5)
    optimizer.step()
    print("Epoch: {}/{}...".format(i+1, epoch),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()))

Epoch: 1/4... Step: 1... Loss: 0.749441...


RuntimeError: ignored